# Modelling metabolism in WHEAT

### Generating WHEAT core model

In [1]:
import pandas as pd
from cobra import io
from cobra.core import Metabolite, Reaction
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "./../plantcoremetabolism-model/PlantCoreMetabolism_v2_1_0.xml"
model = io.read_sbml_model(fname)
model.solver='glpk'
display.clear_output()

#from sweetlovegroup import transform
#model = transform.fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
#rxn = model.reactions.get_by_id("Biomass_tx")
#rxn.id = "Protein_biomass_tx"
#for met in ["K_b","Ca_b","Mg_b"]:
#    met  = model.metabolites.get_by_id(met)
#    coeff = rxn.metabolites.get(met)
#    rxn.add_metabolites({met:-1*coeff})
#met = Metabolite("Protein_b",name="Protein_biomass")
#formula_dict = rxn.check_mass_balance()
#met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
#met.charge = formula_dict["charge"]*-1
#met.compartment = "b"
#rxn.add_metabolites({met:1})
model.reactions.AraCore_Biomass_tx.remove_from_model()

for rxn in model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_bound = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000

#remove charged state of PGPs since they are minor (0.03)
model = removeSpecificMetChargedState(model,["aL_1_PHOSPHATIDYL_GLYCEROL_P_p","aL_1_PHOSPHATIDYL_GLYCEROL_P_m"])

In [2]:
import pandas as pd

df = pd.read_csv("Data/biomass_wheat.csv")

FA=["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p","OLEATE_CPD_p",
    "Octadecadienoate_p","LINOLENIC_ACID_p","ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]



In [3]:
biomass = df.set_index("Unnamed: 0")

### Use biomass composition to generate biomass equations

### Run stem

In [4]:
temp  = model.copy()

for met in temp.reactions.Phloem_output_tx.metabolites.keys():
    met2 = met.copy()
    if met.id=="sSUCROSE_b":
        met2.id = met.id.replace("_b","_ph")
    elif "PROTON" in met.id:
        continue
    else:
        met2.id = met.id.replace("_c","_ph")
    met2.compartment = "ph"
    temp.add_metabolites(met2)
    
    rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
    rxn.add_metabolites({met2:-1,temp.metabolites.get_by_id("PROTON_e"):-1,met:1,temp.metabolites.get_by_id("PROTON_c"):1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    temp.add_reaction(rxn)
    
    print(rxn.reaction)
    

L_ASPARTATE_ph + PROTON_e --> L_ASPARTATE_c + PROTON_c
GLT_ph + PROTON_e --> GLT_c + PROTON_c
PROTON_e + sSUCROSE_ph --> PROTON_c + sSUCROSE_b
PROTON_e + TRP_ph --> PROTON_c + TRP_c
L_ALPHA_ALANINE_ph + PROTON_e --> L_ALPHA_ALANINE_c + PROTON_c
ILE_ph + PROTON_e --> ILE_c + PROTON_c
PROTON_e + VAL_ph --> PROTON_c + VAL_c
GLN_ph + PROTON_e --> GLN_c + PROTON_c
GLC_ph + PROTON_e --> GLC_c + PROTON_c
ASN_ph + PROTON_e --> ASN_c + PROTON_c
CYS_ph + PROTON_e --> CYS_c + PROTON_c
FRU_ph + PROTON_e --> FRU_c + PROTON_c
HIS_ph + PROTON_e --> HIS_c + PROTON_c
GLY_ph + PROTON_e --> GLY_c + PROTON_c
LEU_ph + PROTON_e --> LEU_c + PROTON_c
PHE_ph + PROTON_e --> PHE_c + PROTON_c
PROTON_e + THR_ph --> PROTON_c + THR_c
PROTON_e + SER_ph --> PROTON_c + SER_c
4_AMINO_BUTYRATE_ph + PROTON_e --> 4_AMINO_BUTYRATE_c + PROTON_c
PROTON_e + TYR_ph --> PROTON_c + TYR_c
ARG_ph + PROTON_e --> ARG_c + PROTON_c
MET_ph + PROTON_e --> MET_c + PROTON_c
PROTON_e + PRO_ph --> PROTON_c + PRO_c
LYS_ph + PROTON_e --> LYS_c

In [5]:
# def generateStemModel(model):
#     for met in model.reactions.Phloem_output_tx.metabolites.keys():
#         met2 = met.copy()
#         if met.id=="sSUCROSE_b":
#             met2.id = "SUCROSE_ph"
#             met = model.metabolites.get_by_id("SUCROSE_c")
#         elif "PROTON" in met.id:
#             continue
#         else:
#             met2.id = met.id.replace("_c","_ph")
#         met2.compartment = "ph"
#         model.add_metabolites(met2)
        
#         rxn = Reaction(met2.id+"_exchange")
#         rxn.add_metabolites({met2:1})
#         model.add_reaction(rxn)

#         rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
#         rxn.add_metabolites({met2:-1,model.metabolites.get_by_id("PROTON_e"):-1,
#                              met:1,model.metabolites.get_by_id("PROTON_c"):1})
#         rxn.lower_bound = 0
#         rxn.upper_bound = 1000
#         model.add_reaction(rxn)

#         #print(rxn.reaction)
#     return model

In [6]:
from cobra import flux_analysis
stem_model = model.copy()

k = "stem"

stem_model = updateFAcomposition(stem_model,k,biomass)

rxn = Reaction("Biomass_stem_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    else:
        rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
stem_model.add_reaction(rxn)

stem_model = generateStemModel(stem_model)

for rxn in stem_model.reactions:
    if rxn.id=="SUCROSE_ph_exchange":
        rxn.lower_bound = 1
        rxn.upper_bound = 1
    elif "ph_exchange" in rxn.id:
        rxn.lower_bound = 0
        rxn.upper_bound = 0

stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 0
stem_model.reactions.Sucrose_tx.lower_bound = 0
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1

sol = flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

0.07899508449495768
Biomass flux =0.029372491591109825


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 40.42553191489362 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 77.72340425531915 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 13.829787234042554 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.127659574468085 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 16.757994947514675 is not an integer (in form

In [7]:
for rxn in stem_model.reactions:
    if rxn.objective_coefficient!=0:
        print(rxn.id)

Biomass_stem_tx


In [8]:
stem_model.reactions.SUCROSE_ph_exchange.flux*12

12.0

In [9]:
stem_model.reactions.Biomass_stem_tx.flux*224.21594939662592

6.585781088245101

In [10]:
stem_model.reactions.CO2_tx.flux

-2.5277876782379476

### Run root

In [11]:
# def generateRootModel(model):
#     for met in model.reactions.Phloem_output_tx.metabolites.keys():
#         met2 = met.copy()
#         if met.id=="sSUCROSE_b":
#             met2.id = "SUCROSE_ph"
#             met = model.metabolites.get_by_id("SUCROSE_c")
#         elif "PROTON" in met.id:
#             continue
#         else:
#             met2.id = met.id.replace("_c","_ph")
#         met2.compartment = "ph"
#         model.add_metabolites(met2)
        
#         rxn = Reaction(met2.id+"_exchange")
#         rxn.add_metabolites({met2:1})
#         model.add_reaction(rxn)

#         rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
#         rxn.add_metabolites({met2:-1,model.metabolites.get_by_id("PROTON_e"):-1,
#                              met:1,model.metabolites.get_by_id("PROTON_c"):1})
#         rxn.lower_bound = 0
#         rxn.upper_bound = 1000
#         model.add_reaction(rxn)

#     #add xylem reactions
#     for met in ["CAII","MGII","KI","NITRATE","SULFATE","AMMONIUM","WATER","GLT","L_ASPARTATE","ASN","GLN"]:
#         met2 = model.metabolites.get_by_id(met+"_c").copy()
#         met2.id = met+"_xy"
#         met2.compartment = "xy"

#         rxn = Reaction(met+"_exchange")
#         rxn.add_metabolites({met2:-1})
#         rxn.lower_bound = 0
#         rxn.upper_bound = 1000
#         model.add_reaction(rxn)

#         rxn = Reaction(met+"_xylem_export")
#         rxn.add_metabolites({model.metabolites.get_by_id(met+"_c"):-1,met2:1})
#         model.add_reaction(rxn)
        
#     #adding symbiont compartment
#     from cobra import io
#     rhizo = io.read_sbml_model("Data/iCC541.xml")
#     for met in rhizo.metabolites:
#         met.compartment = met.compartment+"_rhizo"
#     rhizo.compartments={"c_rhizo":"rhizobe cytosol","e_rhizo":"rhizobe extracellular"}
#     model = model+rhizo
    
# #     rxn = Reaction("Sucrose_exchange_symbiont")
# #     rxn.name = rxn.id.replace("_"," ")
# #     rxn.add_metabolites({model.metabolites.get_by_id("SUCROSE_c"):-1,model.metabolites.get_by_id("cpd00076[e0]"):1})
# #     rxn.lower_bound = -1000
# #     rxn.upper_bound = 1000
# #     model.add_reaction(rxn)
    
#     rxn = Reaction("Alanine_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("L_ALPHA_ALANINE_c"):-1,
#                          model.metabolites.get_by_id("ala__L[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     rxn = Reaction("Aspartate_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("L_ASPARTATE_c"):-1,
#                          model.metabolites.get_by_id("asp__L[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     rxn = Reaction("Glutamate_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("GLT_c"):-1,
#                          model.metabolites.get_by_id("glu__L[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     rxn = Reaction("Malate_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("MAL_c"):-1,
#                          model.metabolites.get_by_id("mal__L[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     rxn = Reaction("Succinate_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("SUC_c"):-1,
#                          model.metabolites.get_by_id("succ[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     rxn = Reaction("Ammonium_exchange_symbiont")
#     rxn.name = rxn.id.replace("_"," ")
#     rxn.add_metabolites({model.metabolites.get_by_id("AMMONIUM_c"):-1,
#                          model.metabolites.get_by_id("fixedNH3[e]"):1})
#     rxn.lower_bound = -1000
#     rxn.upper_bound = 1000
#     model.add_reaction(rxn)
    
#     return model

In [12]:
from cobra import flux_analysis
root_model = model.copy()

k = "root"
root_model = updateFAcomposition(root_model,k,biomass)

rxn = Reaction("Biomass_root_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)

root_model = generateRootModel(root_model)

root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 0
root_model.reactions.Sucrose_tx.lower_bound = 0
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0


root_model.reactions.Nitrate_tx.upper_bound = 0
root_model.reactions.Nitrate_tx.lower_bound = 0
root_model.reactions.NH4_tx.upper_bound = 0
root_model.reactions.NH4_tx.lower_bound = 0
 
    
for rxn in root_model.reactions.query("_phloem_"):
    if "SUCROSE" not in rxn.id :
        rxn.lower_bound = 0
        rxn.upper_bound = 0

for rxn in root_model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_bound = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000
    rxn.objective_coefficient=0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux = "+str(root_model.reactions.Biomass_root_tx.flux))

0.07162516963517566


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 16.67231104150657 is not an integer (in formula C16.67231104150657H32.113075347436585O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 32.113075347436585 is not an integer (in formula C16.67231104150657H32.113075347436585O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 37.672311041506575 is not an integer (in formula C37.672311041506575H67.11307534743659N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 67.11307534743659 is not an integer (in formula C37.672311041506575H67.11307534743659N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.9999999999999996 is not an integer (in formula C37.672311041506575H67.11307534743659N7.0O17.0P2.9999999999999996S1.0)


Biomass flux = 0.0


### Run seed

In [13]:
# def generateSeedModel(model):
#     for met in model.reactions.Phloem_output_tx.metabolites.keys():
#         met2 = met.copy()
#         if met.id=="sSUCROSE_b":
#             met2.id = "SUCROSE_ph"
#             met = model.metabolites.get_by_id("SUCROSE_c")
#         elif "PROTON" in met.id:
#             continue
#         else:
#             met2.id = met.id.replace("_c","_ph")
#         met2.compartment = "ph"
#         model.add_metabolites(met2)
        
#         rxn = Reaction(met2.id+"_exchange")
#         rxn.add_metabolites({met2:1})
#         model.add_reaction(rxn)

#         rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
#         rxn.add_metabolites({met2:-1,model.metabolites.get_by_id("PROTON_e"):-1,
#                              met:1,model.metabolites.get_by_id("PROTON_c"):1})
#         rxn.lower_bound = 0
#         rxn.upper_bound = 1000
#         model.add_reaction(rxn)

#         #print(rxn.reaction)
#     return model

In [14]:
from cobra import flux_analysis
seed_model = model.copy()

seed_model = generateSeedModel(seed_model)

k = "seed"
seed_model = updateFAcomposition(seed_model,k,biomass)


for rxn in seed_model.reactions:
    if rxn.id=="SUCROSE_ph_exchange":
        rxn.lower_bound = 1
        rxn.upper_bound = 1
    elif "ph_exchange" in rxn.id:
        rxn.lower_bound = 0
        rxn.upper_bound = 0



rxn = Reaction("Biomass_seed_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 0
seed_model.reactions.Sucrose_tx.lower_bound = 0
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

0.0
Biomass flux =3.5046362552021217


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.9999999999999996 is not an integer (in formula C37.0H66.0N7.0O17.0P2.9999999999999996S1.0)
  warn(


In [15]:
seed_model.reactions.Biomass_seed_tx.flux*796.2867713906156

2790.695488553395

In [16]:
seed_model.reactions.SUCROSE_ph_exchange.flux*12

12.0

In [17]:
seed_model.reactions.CO2_tx.flux

-0.7042271016638176

### Generate diel leaf model



In [18]:
from cobra import flux_analysis
leaf_model = model.copy()

k = "leaf"
leaf_model = updateFAcomposition(leaf_model,k,biomass)

rxn = Reaction("Biomass_leaf_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})

rxn.lower_bound = 0
rxn.upper_bound = 1000
leaf_model.add_reaction(rxn)

leaf_model.reactions.GLC_tx.upper_bound = 0
leaf_model.reactions.GLC_tx.lower_bound = 0
leaf_model.reactions.Sucrose_tx.upper_bound = 0
leaf_model.reactions.Sucrose_tx.lower_bound = 0
leaf_model.reactions.Photon_tx.upper_bound = 0
leaf_model.reactions.Photon_tx.lower_bound = 0
leaf_model.reactions.Biomass_leaf_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(leaf_model)
print("Biomass flux ="+str(leaf_model.reactions.Biomass_leaf_tx.flux))

    


0.06642684859823242
Biomass flux =0.0


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 16.791156303205355 is not an integer (in formula C16.791156303205355H31.636917070762998O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 31.636917070762998 is not an integer (in formula C16.791156303205355H31.636917070762998O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 37.791156303205355 is not an integer (in formula C37.791156303205355H66.636917070763N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 66.636917070763 is not an integer (in formula C37.791156303205355H66.636917070763N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.9999999999999996 is not an integer (in formula C37.791156303205355H66.636917070763N7.0O17.0P2.9999999999999996S1.0)
  war

In [19]:
from sweetlovegroup.transform import setupC3DielModel
leaf_model = setupC3DielModel(leaf_model)

0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
bHIS_v1 --> bHIS_v2
0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2


In [20]:
leaf_model.reactions.Phloem_output_tx1.objective_coefficient = 0
leaf_model.reactions.Phloem_output_tx2.objective_coefficient = 0

leaf_model.reactions.Photon_tx1.upper_bound = 500
leaf_model.reactions.Photon_tx1.lower_bound = 500

leaf_model.reactions.Biomass_leaf_tx1.objective_coefficient = 0
leaf_model.reactions.Biomass_leaf_tx1.upper_bound=0
leaf_model.reactions.Biomass_leaf_tx1.lower_bound=0
leaf_model.reactions.Biomass_leaf_tx2.upper_bound=0
leaf_model.reactions.Biomass_leaf_tx2.lower_bound=0


from cobra.flux_analysis import pfba
sol = pfba(leaf_model)
print("Phloem export rate ="+str(sol.fluxes["diel_biomass"]*4))

Phloem export rate =3.5504474309760568


In [28]:
growing_leaf_model = leaf_model.copy()

growing_leaf_model.reactions.Biomass_leaf_tx1.upper_bound=1000
growing_leaf_model.reactions.Biomass_leaf_tx2.upper_bound=1000

met1=Metabolite("Leaf_biomass1",compartment="c1")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx1
rxn.add_metabolites({met1:1})

met2=Metabolite("Leaf_biomass2",compartment="c2")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx2
rxn.add_metabolites({met2:1})

growing_leaf_model.reactions.diel_biomass.add_metabolites({growing_leaf_model.metabolites.X_Phloem_contribution_t1:3,
                                                           growing_leaf_model.metabolites.X_Phloem_contribution_t2:1,
                                                           met1:-0.75, met2:-0.25})


from cobra.flux_analysis import pfba
sol = pfba(growing_leaf_model)
print("Biomass accumulation rate rate ="+str(sol.fluxes["diel_biomass"]))

Biomass accumulation rate rate =0.18296995332247598


In [23]:
io.write_sbml_model(root_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\wheat_root_model.xml")

In [24]:
io.write_sbml_model(stem_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\wheat_stem_model.xml")

In [25]:
io.write_sbml_model(seed_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\wheat_seed_model.xml")

In [26]:
io.write_sbml_model(leaf_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\wheat_sourceleaf_model.xml")

In [29]:
io.write_sbml_model(growing_leaf_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\wheat_sinkleaf_model.xml")